In [1]:
import pandas as pd
import zipfile
import geopandas as gpd
import numpy as np

In [2]:
c = pd.read_csv('../data_processed/COM_VIO_2014_2016_BBL_WEEK_AVG_onlyheatingseason.csv')
c = c.drop('Unnamed: 0',axis=1)
a = pd.read_csv('../data_processed/BBL_ACS_2016.csv')
a = a.drop('Unnamed: 0',axis=1)

In [3]:
c.head(2)

,BBL,com_count,vio_count
0,1000047501,0.031915,0.0
1,1000100032,0.010638,0.0


In [4]:
a.head(2)

,BBL,CBG,gas_utility_r,avg_bldng_age,ownership_r,year,pop,female_r,elderly_r,white_r,...,hh_child_6_r,hh_living_alone_r,edu_high_r,non_eng_r,unemployed_r,vacancy_r,med_rent,med_income,merried_r,longercommute_r
0,5080500076,360850248001,0.795322,54.249527,0.586745,2016.0,1373.0,0.441369,0.09177,0.962855,...,0.053097,0.284600,0.210817,0.019493,0.042266,0.030246,980.0,51992.0,0.485255,0.446529
1,5079390055,360850244011,0.876565,35.051878,0.803220,2016.0,1698.0,0.510012,0.14311,0.959364,...,0.423780,0.187835,0.262467,0.000000,0.035823,0.000000,1259.0,84967.0,0.573906,0.450751


In [5]:
print 'length complaints df: ' + str(len(c))
print 'length acs df: ' + str(len(a))

length complaints df: 53310
length acs df: 857237


In [6]:
pluto = pd.read_csv('../data_processed/pluto_14')
pluto = pluto.drop('Unnamed: 0',axis=1)
pluto.head(2)

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (27,37,40,46,53,64,65,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,APPBBL,APPDate,Address,AllZoning1,AllZoning2,AreaSource,AssessLand,AssessTot,BBL,BldgArea,...,YearAlter2,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry
0,0.0,NaN,437 PURDY AVENUE,R3-1,NaN,7,15120.0,23400.0,5.007470e+09,1488,...,0,1960,NaN,10314,R3-1,NaN,NaN,NaN,20d,"POLYGON ((944681.4200000018 161477.381400004, ..."
1,0.0,NaN,CUNARD AVENUE,R3A/HS,NaN,4,21518.0,21518.0,5.006230e+09,0,...,0,0,NaN,10304,R3A,NaN,NaN,NaN,21d,"POLYGON ((959872.0577999949 162992.5866000056,..."


In [7]:
pluto = pluto[pluto['UnitsRes'] != 0]

In [8]:
pluto['BBL'] = pluto['BBL'].astype(int)

### hypothesis: problem with merge isn't that the dataset is insufficient, it's that there are about 1500 bbls without any residential units that have heat/hot water complaints or violations!

In [9]:
pluto['AssessSqft'] = pluto['AssessTot'] / pluto['BldgArea']

In [10]:
var = []
i=0
for elem in pluto['BldgClass']:
    if elem[i][0] == 'A':
        var.append(1)
    elif elem[i][0] == 'B':
        var.append(2)
    elif elem[i][0] == 'C':
        var.append(3)
    elif elem[i][0] == 'S':
        var.append(3)
    elif elem[i][0] == 'D':
        var.append(4)
    elif elem[i] == 'O8':
        var.append(4)
    elif elem[i][0] == 'L':
        var.append(6)
    elif elem[i][0] == 'R':
        var.append(5)
    else:
        var.append(0)
pluto['bldgclass_updated'] = var

In [11]:
pluto = pluto.drop(['APPBBL', 'APPDate', 'Address', 'AreaSource', 'BldgDepth', 'BldgFront', \
                                   'Block', 'BoroCode', 'Borough', 'CB2010', 'CD', 'CT2010', 'ComArea', \
                                   'CommFAR', 'CondoNo', 'Council', 'EDesigNum', 'FacilFAR', 'FactryArea', \
                                   'FireComp', 'GarageArea', 'HealthArea', 'HistDist', 'IrrLotCode', 'Landmark', \
                                   'Lot', 'LotArea', 'LotDepth', 'LotFront', 'LtdHeight', 'MAPPLUTO_F', \
             'OfficeArea', 'OtherArea', 'Overlay1', 'Overlay2', 'PLUTOMapID', 'PolicePrct', 'ResidFAR', \
             'RetailArea', 'Sanborn', 'SchoolDist', 'StrgeArea', 'TaxMap', 'Tract2010', 'YearAlter1', \
             'YearAlter2', 'ZoneDist1', 'ZipCode', 'ZoneDist2', 'ZoneDist3', 'ZoneDist4', 'ZoneMap', \
             'XCoord', 'Version', 'YCoord', 'ZMCode', 'SPDist1', 'SPDist2', 'ExemptTot', 'BuiltCode', \
             'AllZoning1', 'AllZoning2', 'ExemptLand', 'geometry', 'Easements', 'SHAPE_Area', 'SHAPE_Leng', \
             'OwnerName', 'Ext', 'SplitZone', 'BldgClass', 'AssessTot', 'BldgArea', 'UnitsTotal', 'OwnerType', \
                   'AssessLand'], axis=1)

In [12]:
pluto.head(2)

,BBL,BsmtCode,BuiltFAR,LandUse,LotType,NumBldgs,NumFloors,ProxCode,ResArea,UnitsRes,YearBuilt,AssessSqft,bldgclass_updated
0,5007470028,2.0,0.35,1.0,5.0,1,1.0,1.0,1488,2,1960,15.725806,2
2,5007130017,2.0,0.31,1.0,5.0,2,2.5,1.0,1316,1,1920,17.174772,1


In [13]:
pluto.isnull().sum()

BBL                   0
BsmtCode              0
BuiltFAR              0
LandUse              13
LotType               0
NumBldgs              0
NumFloors             0
ProxCode              0
ResArea               0
UnitsRes              0
YearBuilt             0
AssessSqft           11
bldgclass_updated     0
dtype: int64

In [14]:
cp = pd.merge(c, pluto, on='BBL', how='left')
cpa = pd.merge(cp, a, how='left')

In [15]:
print len(cp)
print len(cpa)

53310
53310


In [16]:
cpa.isnull().sum()

BBL                     0
com_count               0
vio_count               0
BsmtCode             1528
BuiltFAR             1528
LandUse              1528
LotType              1528
NumBldgs             1528
NumFloors            1528
ProxCode             1528
ResArea              1528
UnitsRes             1528
YearBuilt            1528
AssessSqft           1531
bldgclass_updated    1528
CBG                    36
gas_utility_r          36
avg_bldng_age          36
ownership_r            36
year                   36
pop                    36
female_r               36
elderly_r              36
white_r                36
black_r                36
asian_r                36
hh                     36
hh_child_6_r           36
hh_living_alone_r      36
edu_high_r             36
non_eng_r              36
unemployed_r           36
vacancy_r              36
med_rent               36
med_income             36
merried_r              36
longercommute_r        36
dtype: int64

In [17]:
cpa.to_csv('../data_processed/acs_bbl_complaints')

In [18]:
pluto2 = pd.read_csv('../data_processed/pluto_14')
pluto2 = pluto2.drop('Unnamed: 0',axis=1)
pluto2 = pluto2[pluto2['ResArea'] != 0]
pluto2['BBL'] = pluto2['BBL'].astype(int)
pluto2['AssessSqft'] = pluto2['AssessTot'] / pluto2['BldgArea']
pluto2 = pluto2.dropna(subset=['BldgClass'])
var = []
i=0
for elem in pluto2['BldgClass']:
    if elem[i][0] == 'A':
        var.append(1)
    elif elem[i][0] == 'B':
        var.append(2)
    elif elem[i][0] == 'C':
        var.append(3)
    elif elem[i][0] == 'S':
        var.append(3)
    elif elem[i][0] == 'D':
        var.append(4)
    elif elem[i] == 'O8':
        var.append(4)
    elif elem[i][0] == 'L':
        var.append(6)
    elif elem[i][0] == 'R':
        var.append(5)
    else:
        var.append(0)
pluto2['bldgclass_updated'] = var
pluto2 = pluto2.drop(['APPBBL', 'APPDate', 'Address', 'AreaSource', 'BldgDepth', 'BldgFront', \
                                   'Block', 'BoroCode', 'Borough', 'CB2010', 'CD', 'CT2010', 'ComArea', \
                                   'CommFAR', 'CondoNo', 'Council', 'EDesigNum', 'FacilFAR', 'FactryArea', \
                                   'FireComp', 'GarageArea', 'HealthArea', 'HistDist', 'IrrLotCode', 'Landmark', \
                                   'Lot', 'LotArea', 'LotDepth', 'LotFront', 'LtdHeight', 'MAPPLUTO_F', \
             'OfficeArea', 'OtherArea', 'Overlay1', 'Overlay2', 'PLUTOMapID', 'PolicePrct', 'ResidFAR', \
             'RetailArea', 'Sanborn', 'SchoolDist', 'StrgeArea', 'TaxMap', 'Tract2010', 'YearAlter1', \
             'YearAlter2', 'ZoneDist1', 'ZipCode', 'ZoneDist2', 'ZoneDist3', 'ZoneDist4', 'ZoneMap', \
             'XCoord', 'Version', 'YCoord', 'ZMCode', 'SPDist1', 'SPDist2', 'ExemptTot', 'BuiltCode', \
             'AllZoning1', 'AllZoning2', 'ExemptLand', 'geometry', 'Easements', 'SHAPE_Area', 'SHAPE_Leng', \
             'OwnerName', 'Ext', 'SplitZone', 'BldgClass', 'AssessTot', 'BldgArea', 'UnitsTotal', 'OwnerType', 
                     'AssessLand'], axis=1)
cp2 = pd.merge(c, pluto2, on='BBL', how='left')
cpa2 = pd.merge(cp2, a, how='left')
print len(cp2)
print len(cpa2)
print cpa2.isnull().sum()

53310
53310
BBL                     0
com_count               0
vio_count               0
BsmtCode             1660
BuiltFAR             1660
LandUse              1660
LotType              1660
NumBldgs             1660
NumFloors            1660
ProxCode             1660
ResArea              1660
UnitsRes             1660
YearBuilt            1660
AssessSqft           1660
bldgclass_updated    1660
CBG                    36
gas_utility_r          36
avg_bldng_age          36
ownership_r            36
year                   36
pop                    36
female_r               36
elderly_r              36
white_r                36
black_r                36
asian_r                36
hh                     36
hh_child_6_r           36
hh_living_alone_r      36
edu_high_r             36
non_eng_r              36
unemployed_r           36
vacancy_r              36
med_rent               36
med_income             36
merried_r              36
longercommute_r        36
dtype: int64


In [19]:
cpa = cpa.dropna()

In [21]:
cpa.head()

,BBL,com_count,vio_count,BsmtCode,BuiltFAR,LandUse,LotType,NumBldgs,NumFloors,ProxCode,...,hh_child_6_r,hh_living_alone_r,edu_high_r,non_eng_r,unemployed_r,vacancy_r,med_rent,med_income,merried_r,longercommute_r
1,1000100032,0.010638,0.00000,2.0,4.70,4.0,5.0,1.0,5.0,3.0,...,0.504274,0.337731,0.755677,0.030343,0.023172,0.312160,3600.0,147500.0,0.493571,0.149542
2,1000150022,0.010638,0.00000,5.0,24.93,3.0,3.0,1.0,31.0,0.0,...,0.000000,0.569112,0.905780,0.020077,0.037861,0.262948,2715.0,123832.0,0.274537,0.083185
3,1000160015,0.010638,0.00000,5.0,7.68,3.0,5.0,1.0,9.0,0.0,...,0.342282,0.286491,0.745763,0.032609,0.051647,0.174888,2954.0,132037.0,0.608503,0.298693
4,1000160020,0.021277,0.00000,5.0,7.77,3.0,5.0,1.0,9.0,0.0,...,0.342282,0.286491,0.745763,0.032609,0.051647,0.174888,2954.0,132037.0,0.608503,0.298693
5,1000160100,0.042553,0.56383,5.0,8.52,4.0,5.0,6.0,5.0,0.0,...,0.369863,0.452632,0.784270,0.064912,0.000000,0.113530,2563.0,130294.0,0.576404,0.229050


In [22]:
len(cpa)

51770

In [48]:
order = [elem for elem in cpa.columns]

In [50]:
order = [[order[0]], order[2:], [order[1]]]

In [52]:
f = [y for x in order for y in x]

In [55]:
cpa = cpa[f]

In [57]:
cpa.head()

,BBL,vio_count,BsmtCode,BuiltFAR,LandUse,LotType,NumBldgs,NumFloors,ProxCode,ResArea,...,hh_living_alone_r,edu_high_r,non_eng_r,unemployed_r,vacancy_r,med_rent,med_income,merried_r,longercommute_r,com_count
1,1000100032,0.00000,2.0,4.70,4.0,5.0,1.0,5.0,3.0,3474.0,...,0.337731,0.755677,0.030343,0.023172,0.312160,3600.0,147500.0,0.493571,0.149542,0.010638
2,1000150022,0.00000,5.0,24.93,3.0,3.0,1.0,31.0,0.0,331746.0,...,0.569112,0.905780,0.020077,0.037861,0.262948,2715.0,123832.0,0.274537,0.083185,0.010638
3,1000160015,0.00000,5.0,7.68,3.0,5.0,1.0,9.0,0.0,230765.0,...,0.286491,0.745763,0.032609,0.051647,0.174888,2954.0,132037.0,0.608503,0.298693,0.010638
4,1000160020,0.00000,5.0,7.77,3.0,5.0,1.0,9.0,0.0,230765.0,...,0.286491,0.745763,0.032609,0.051647,0.174888,2954.0,132037.0,0.608503,0.298693,0.021277
5,1000160100,0.56383,5.0,8.52,4.0,5.0,6.0,5.0,0.0,1727692.0,...,0.452632,0.784270,0.064912,0.000000,0.113530,2563.0,130294.0,0.576404,0.229050,0.042553


In [59]:
cpa.to_csv('../data_processed/acs_bbl_complaints')